# Decorators  in Python

## Functions as arguments

Like all objects, functions can be arguments to functions

In [12]:
def add(x,y):
    return x+y

def sub(x, y):
    return x-y

def apply(func, x, y):
    return func(x, y)

In [13]:
apply(add, 1, 2)

3

In [14]:
apply(sub, 7, 5)

2

## Functions inside functions

Python allows nested function definitions:

In [15]:
def g(x, y):
    
    def cube(x):
        return x*x*x
    
    return y*cube(x)

g(4, 6)

384

## Function returning functions

In [21]:
def h():
    pi = 0.13
    def inner_h():
        print("Inside inner_h but can access pi={}".format(pi))
        
    return inner_h

foo = h()
foo

<function __main__.h.<locals>.inner_h>

In [20]:
foo()

Inside inner_h but can access pi='0.13'


## More functions returning functions: *decorators*

A toy example

In [1]:
def foo():
    return 1

def outer(func):
    def inner():
        print("before calling func")
        return func()
    return inner

In [2]:
decorated = outer(foo)

The function `decorated` is a decorated version of function `foo`.
It is `foo` plus something more:

In [3]:
decorated()

before calling func


1

To simplify, we could just write
```python 
foo = outer(foo)
```
to replace foo with its decorated version each time it is called

## A (slightly) more useful decorator

Suppose we have been given a function that only works for some numerical inputs:

In [5]:
from math import log
def f(x):
    return log(x) - 2

In [6]:
f(5)

-0.3905620875658997

In [7]:
f(-1)

ValueError: math domain error

Suppose we want to limit the range of values sent to this function:

The idea is that we **wrap** the function inside another function:

In [28]:
def checkrange(func):
    def inner(x, limit):
        if x <= limit:
            print("out of range")
        else:
            return func(x)
    return inner

In [32]:
fcheck = checkrange(f)
fcheck(5, limit=6)

out of range


In [10]:
fcheck(-1)

out of range


Voilà!!

## The `@decorator` syntax

Python provides a short notation for decorating a function with
another function:

In [15]:
@checkrange
def g(x):
    return log(x) - 2

In [16]:
g(0)

out of range


This is essentially the same as writing `g = checkrange(g)`.

A decorator is simply a function taking another function as input
and returning another function. 

The syntax `@decorator` is a
short-cut for the more explicit `f = decorator(f)`.

## A (much) more useful decorator: memoization

Assume we have a slow function. Something like

In [17]:
from time import sleep

def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

In [18]:
%timeit slow(1, 2)

1 s ± 145 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


We call the function with the same input arguments, and hence perform the same (slow) calculations multiple times.

The idea of memoization (or buffering) is to buffer the input-output pairs for which the function was called.
If the function is called twice with same input arguments, we return the buffer value.

The implementation of a memoization with a `decorator` could look like:

In [20]:
cache = {}  # Stores all input-output pairs

def memoize(func):
    ''' Caches a function's return value each time it is called.
        If called later with the same arguments, the cached value is returned
        (not reevaluated). '''
    
    def inner(x, y):
        if (x, y) in cache:
            return cache[(x, y)]
        else:
            result = func(x, y)
            cache[(x, y)] = result
            return result
        
    return inner

Now we can apply the decorator to our slow function

In [21]:
@memoize
def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

... and test it out

In [27]:
slow(1, 2)

2

## Decorator summary 

* A function that takes a function as argument and returns a modified function
* `@decorator` syntax simply a short cut for the standard function call `f = decorator(f)`.